# Setup

Setup the environment

In [0]:
catalog = "project"
br_schema = dbName = db = "bronze"
sl_schema =  "silver"
gd_schema =  "gold"

data_volume_name = "source_data"
test_volume_name = "test_data"

In [0]:
## Set up Catalog, schemas and volumes
spark.sql(f'CREATE CATALOG IF NOT EXISTS `{catalog}`')
spark.sql(f'USE CATALOG `{catalog}`')
spark.sql(f'CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{gd_schema}`')
spark.sql(f'CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{sl_schema}`')
spark.sql(f'CREATE SCHEMA IF NOT EXISTS `{catalog}`.`{br_schema}`')
spark.sql(f'USE SCHEMA `{br_schema}`')
spark.sql(f'CREATE VOLUME IF NOT EXISTS `{catalog}`.`{br_schema}`.`{data_volume_name}`')
data_volume_folder =  f"/Volumes/{catalog}/{db}/{data_volume_name}"
spark.sql(f'CREATE VOLUME IF NOT EXISTS `{catalog}`.`{br_schema}`.`{test_volume_name}`')
test_volume_folder =  f"/Volumes/{catalog}/{db}/{test_volume_name}"

In [0]:
import shutil
import glob
import os

# Copy the test_data.csv file
for file in glob.glob("../fixtures/test*.csv"):
    shutil.copy(file, test_volume_folder + "/")

# Copy the data_group*.csv files
for file in glob.glob("../fixtures/data_group*.csv"):
    shutil.copy(file, data_volume_folder + "/")

In [0]:
data = [
 # tablename         quality            name              constraint
 ("turbine_data",   "bronze",   "correct_schema", "_rescued_data IS NULL"),
 ("turbine_data",   "bronze",   "valid_id", "turbine_id IS NOT NULL AND turbine_id > 0"),
 ("turbine_data",   "bronze",   "timestamp",    "timestamp IS NOT NULL AND timestamp < current_timestamp()"),
 ("turbine_data",   "silver",    "wind_speed",  "windspeed IS NOT NULL AND windspeed > 0"),
 ("turbine_data",   "silver",    "wind_direction",  "winddirection IS NOT NULL AND winddirection < 360")
]

spark.createDataFrame(data=data, schema=["tablename", "quality","name", "constraint"]).write.mode("overwrite").saveAsTable(f"{catalog}.{br_schema}.expectations")